# Chaos Mesh with CTK & RTK

For a Chaos Mesh intro see: [Chaos Mesh docs](https://chaos-mesh.org/docs/).

Chaos Mesh run on Kubernetes and includes fualt injection methods in in Pod, network, file system, and even the kernel.

Chaos Mesh triggers a failt by running kube apply on a YAML file which define the fault injection you want ot run. We have used it in the past by getting Chaos Toolkit to trigger the kube apply as ab experiments method step.

Here I am going to look at Network Fault injection with Chaos Mesh and look at CTK / RTK integration.

## First on minicube

* List hte current kubectl contexts:

In [ ]:
!kubectl config get-contexts

CURRENT   NAME                                                                    CLUSTER                                                                 AUTHINFO                                                                NAMESPACE
          NAME                                                                                                                                                                                                                    chaosiq-console
          gke_chaosiq-prod-20180922_us-west1-a_cluster-prod-20180922-a            gke_chaosiq-prod-20180922_us-west1-a_cluster-prod-20180922-a            gke_chaosiq-prod-20180922_us-west1-a_cluster-prod-20180922-a            
          gke_chaosiq-saas-staging_europe-west2-a_cluster-kind-rabbit             gke_chaosiq-saas-staging_europe-west2-a_cluster-kind-rabbit             gke_chaosiq-saas-staging_europe-west2-a_cluster-kind-rabbit             
          gke_chaosiq-staging-20191115-b4b1f_us-central1_cluster-sel

* Ensure kubectl is connected to minikube

In [ ]:
!kubectl config use-context minikube

Switched to context "minikube".


# Install Chaos Mesh on minkube

Chaos Mesh install from: [Chaos Mesh Getting Started Minikube](https://chaos-mesh.org/docs/installation/get_started_on_minikube)



In [ ]:
!minikube start --kubernetes-version v1.15.0 --cpus 4 --memory "8192mb"

😄  minikube v1.12.1 on Darwin 10.15.5
💥  You have selected Kubernetes 1.15.0, but the existing cluster is running Kubernetes 1.17.3
💣  Non-destructive downgrades are not supported, but you can proceed with one of the following options:

  1) Recreate the cluster with Kubernetes 1.15.0, by running:

    minikube delete
    minikube start --kubernetes-version=v1.15.0

  2) Create a second cluster with Kubernetes 1.15.0, by running:

    minikube start -p minikube2 --kubernetes-version=v1.15.0

  3) Use the existing cluster at version Kubernetes 1.17.3, by running:

    minikube start --kubernetes-version=v1.17.3
    


In [ ]:
!minikube delete

🔥  Deleting "minikube" in hyperkit ...
💀  Removed all traces of the "minikube" cluster.


In [ ]:
!minikube start --kubernetes-version v1.15.0 --cpus 4 --memory "8192mb"

😄  minikube v1.12.1 on Darwin 10.15.5
🎉  minikube 1.12.2 is available! Download it: https://github.com/kubernetes/minikube/releases/tag/v1.12.2
💡  To disable this notice, run: 'minikube config set WantUpdateNotification false'

✨  Automatically selected the docker driver. Other choices: hyperkit, virtualbox
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔥  Creating docker container (CPUs=4, Memory=8192MB) ...
🐳  Preparing Kubernetes v1.15.0 on Docker 19.03.2 ...
🔎  Verifying Kubernetes components...
🌟  Enabled addons: default-storageclass, storage-provisioner
🏄  Done! kubectl is now configured to use "minikube"

❗  /usr/local/bin/kubectl is version 1.18.6, which may be incompatible with Kubernetes 1.15.0.
💡  You can also use 'minikube kubectl -- get pods' to invoke a matching version


Install helm (use the manual steps from here: [chaos mesh install helm
](https://chaos-mesh.org/docs/installation/installation#install-by-helm://))


In [ ]:
!git clone https://github.com/chaos-mesh/chaos-mesh.git

Cloning into 'chaos-mesh'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 7397 (delta 20), reused 41 (delta 10), pack-reused 7307
Receiving objects: 100% (7397/7397), 42.36 MiB | 4.45 MiB/s, done.
Resolving deltas: 100% (4417/4417), done.


In [ ]:
cd chaos-mesh/

/Users/grant/dev/try/jupyter-one/chaos-mesh


## Step2: Create custom resource type#
To use Chaos Mesh, you must create the related custom resource type first.

In [ ]:
!kubectl apply -f manifests/crd.yaml

customresourcedefinition.apiextensions.k8s.io/iochaos.chaos-mesh.org created
customresourcedefinition.apiextensions.k8s.io/kernelchaos.chaos-mesh.org created
customresourcedefinition.apiextensions.k8s.io/networkchaos.chaos-mesh.org created
customresourcedefinition.apiextensions.k8s.io/podchaos.chaos-mesh.org created
customresourcedefinition.apiextensions.k8s.io/stresschaos.chaos-mesh.org created
customresourcedefinition.apiextensions.k8s.io/timechaos.chaos-mesh.org created


# Install chaos meshStep 3: Install Chaos Mesh
Depending on your environment, there are two methods of installing Chaos Mesh:

Install in Docker environment (this is the case here)


1.   Create the chaos-testing namespace
2.   Install Chaos Mesh using helm:



In [ ]:
!kubectl create ns chaos-testing

namespace/chaos-testing created


In [ ]:
!helm install chaos-mesh helm/chaos-mesh --namespace=chaos-testing

NAME: chaos-mesh
LAST DEPLOYED: Wed Aug  5 15:33:58 2020
NAMESPACE: chaos-testing
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
1. Make sure chaos-mesh components are running
   kubectl get pods --namespace chaos-testing -l app.kubernetes.io/instance=chaos-mesh


In [ ]:
!kubectl get pods --namespace chaos-testing -l app.kubernetes.io/instance=chaos-mesh


NAME                                        READY   STATUS    RESTARTS   AGE
chaos-controller-manager-55f8b565d8-ngtrk   1/1     Running   0          54s
chaos-daemon-kpvzt                          1/1     Running   0          54s


# Run a Chaos Experiment

Now that you have deployed Chaos Mesh in your environment, it's time to use it for your chaos experiments. This document walks you through the process of running chaos experiments. It also describes the regular operations on chaos experiments.

## Step 1: Deploy the target cluster

Chaos mesh has a Webshow example they use for getting started.


In [ ]:
!./examples/web-show/deploy.sh

service/web-show created
deployment.apps/web-show created
Waiting for pod running
Waiting for pod running
Waiting for pod running


Some dificulty connecting to the Web show container after running the above script but looking at the bottom of the deploy.sh, there is a command to port forward to the container using:
```
nohup kubectl port-forward --address 0.0.0.0 svc/web-show 8081:8081 >/dev/null 2>&1 &
```


In [ ]:
!kubectl port-forward --address 0.0.0.0 svc/web-show 8081:8081

Forwarding from 0.0.0.0:8081 -> 8081
Handling connection for 8081
Handling connection for 8081
Handling connection for 8081


Note: Output not shown from above as command was run in a local shell, which has already bound the port

## Step 2: Define the experiment configuration file
The chaos experiment configuration is defined in a YAML file. You need to create your own experiment configuration file based on the available fields in the sample below:
```
apiVersion: chaos-mesh.org/v1alpha1
kind: NetworkChaos
metadata:
  name: web-show-network-delay
spec:
  action: delay # the specific chaos action to inject
  mode: one # the mode to run chaos action; supported modes are one/all/fixed/fixed-percent/random-max-percent
  selector: # pods where to inject chaos actions
    namespaces:
      - default
    labelSelectors:
      "app": "web-show"  # the label of the pod for chaos injection
  delay:
    latency: "10ms"
  duration: "30s" # duration for the injected chaos experiment
  scheduler: # scheduler rules for the running time of the chaos experiments about pods.
    cron: "@every 60s"

```


In [ ]:
!kubectl apply -f ./examples/web-show/network-delay.yaml

networkchaos.chaos-mesh.org/web-show-network-delay created


try  kubectl apply -f ./examples/web-show/network-delay.yaml --v=8

![Web show with netwrok delay](https://drive.google.com/uc?export=view&id=1n-8sljHfWDtgfjtexlhPlsiudX5s0aoV)

Pause the experiment:

```kubectl annotate networkchaos web-show-network-delay experiment.chaos-mesh.org/pause=true```

In [ ]:
!kubectl annotate networkchaos web-show-network-delay experiment.chaos-mesh.org/pause=true

networkchaos.chaos-mesh.org/web-show-network-delay annotated


In [ ]:
!kubectl annotate networkchaos web-show-network-delay experiment.chaos-mesh.org/pause-

networkchaos.chaos-mesh.org/web-show-network-delay annotated


In [ ]:
!kubectl delete -f ./examples/web-show/network-delay.yaml

networkchaos.chaos-mesh.org "web-show-network-delay" deleted


In [ ]:
!minikube stop

✋  Stopping "minikube" in docker ...
🛑  Powering off "minikube" via SSH ...
🛑  Node "minikube" stopped.


Chaos Messh works by applying a manifest using kubectl apply passing a yaml file.

To integratre with CTK/RTK this can be done manually by running an experiment locally that runs the kubectl apply -f as the experiments method step. This give you a manual means to ensure your manifest are working in your environment.

(Add code example to show this)


Info Links
* [Command line Chaos](https://github.com/pingcap/tipocket/blob/master/doc/command_line_chaos.md)
* [about chaos Mesh](https://pingcap.com/blog/chaos-mesh-your-chaos-engineering-solution-for-system-resiliency-on-kubernetes)

In [ ]:
!minikube start

😄  minikube v1.12.1 on Darwin 10.15.5
🆕  Kubernetes 1.18.3 is now available. If you would like to upgrade, specify: --kubernetes-version=v1.18.3
✨  Using the docker driver based on existing profile
👍  Starting control plane node minikube in cluster minikube
🔄  Restarting existing docker container for "minikube" ...
🐳  Preparing Kubernetes v1.15.0 on Docker 19.03.2 ...
🔎  Verifying Kubernetes components...
🌟  Enabled addons: default-storageclass, storage-provisioner
🏄  Done! kubectl is now configured to use "minikube"

❗  /usr/local/bin/kubectl is version 1.18.6, which may be incompatible with Kubernetes 1.15.0.
💡  You can also use 'minikube kubectl -- get pods' to invoke a matching version


In [ ]:
!kubectl get pods --all-namespaces

NAMESPACE       NAME                                        READY   STATUS    RESTARTS   AGE
chaos-testing   chaos-controller-manager-55f8b565d8-ngtrk   1/1     Running   1          17h
chaos-testing   chaos-daemon-kpvzt                          1/1     Running   1          17h
default         web-show-c787785d6-xfqqd                    1/1     Running   1          17h
kube-system     coredns-5c98db65d4-hbd45                    1/1     Running   3          18h
kube-system     etcd-minikube                               1/1     Running   1          18h
kube-system     kube-apiserver-minikube                     1/1     Running   1          18h
kube-system     kube-controller-manager-minikube            1/1     Running   0          9m3s
kube-system     kube-proxy-kxrgd                            1/1     Running   1          18h
kube-system     kube-scheduler-minikube                     1/1     Running   1          18h
kube-system     storage-provisioner                         1/1     R

## Chaos Toolkit extension

Chaos toolkit already has a [Kubernetes extension](https://chaostoolkit.org/extensions/kubernetes). This does not currently support apply_manifest_object but one can be added, that should mean an mififest  can be applied but more specifically if Chaos Mesh is installed in the cluster Chaos Mesh manifests can be applied as part of the experiment.

## Python Kubernetes Lib

The CTK extension uses the python kubernetes library. Just to verify manifest's can be applied using the library lets try that first.

### Example 1 - List Pods



In [ ]:
pip install kubernetes

  Using cached kubernetes-11.0.0-py3-none-any.whl (1.5 MB)
  Using cached websocket_client-0.57.0-py2.py3-none-any.whl (200 kB)
  Using cached google_auth-1.20.0-py2.py3-none-any.whl (91 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached rsa-4.6-py3-none-any.whl (47 kB)
  Using cached cachetools-4.1.1-py3-none-any.whl (10 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
ret = v1.list_pod_for_all_namespaces(watch=False)
for i in ret.items:
    print("%s\t%s\t%s" % (i.status.pod_ip, i.metadata.namespace, i.metadata.name))

Listing pods with their IPs:
172.18.0.4	chaos-testing	chaos-controller-manager-55f8b565d8-ngtrk
172.18.0.3	chaos-testing	chaos-daemon-kpvzt
172.18.0.5	default	web-show-c787785d6-xfqqd
172.18.0.2	kube-system	coredns-5c98db65d4-hbd45
172.17.0.2	kube-system	etcd-minikube
172.17.0.2	kube-system	kube-apiserver-minikube
172.17.0.2	kube-system	kube-controller-manager-minikube
172.17.0.2	kube-system	kube-proxy-kxrgd
172.17.0.2	kube-system	kube-scheduler-minikube
172.17.0.2	kube-system	storage-provisioner


Issue: The problem using the python kubernete library is you can apply a custom crd via the apply api. Used a python implementation of the kube apply command from [this stackoverfow](https://stackoverflow.com/questions/36307950/kubernetes-api-call-equivalent-to-kubectl-apply), which works for a standard manifest (pod deploy), but not for a customer manifest such as the chaos-mesh network delay. The kube aplloy code can be seen below:

In [ ]:
#!/usr/bin/python3
# coding: utf-8
# __________   ________________________________________________   #
# kube_apply - apply Yaml similar to kubectl apply -f file.yaml   #
#                                                                 #
# (C) 2019 Hermann Vosseler <Ichthyostega@web.de>                 #
# This is OpenSource software; licensed under Apache License v2+  #
# ############################################################### #
'''
Utility for the official Kubernetes python client: apply Yaml data.
While still limited to some degree, this utility attempts to provide
functionality similar to `kubectl apply -f`
- load and parse Yaml
- try to figure out the object type and API to use
- figure out if the resource already exists, in which case
  it needs to be patched or replaced alltogether.
- otherwise just create a new resource.

Based on inspiration from `kubernetes/utils/create_from_yaml.py`

@since: 2/2019
@author: Ichthyostega
'''

import pdb
import re
import yaml
import logging

import kubernetes.client
from kubernetes import client, config


def runUsageExample():
    ''' demonstrate usage by creating a simple Pod through default client
    '''
    logging.basicConfig(level=logging.DEBUG)
    #
#   KUBECONFIG = '/path/to/special/kubecfg.yaml'
#   client = kubernetes.config.new_client_from_config(config_file=KUBECONFIG)
#   # --or alternatively--
#   kubernetes.config.load_kube_config(config_file=KUBECONFIG)
    config.load_kube_config()
    k8s_client = client.ApiClient()

    yaml1 ='''
    kind: Pod
apiVersion: v1
metadata:
  name: dummy-pod
  labels:
    blow: job
spec:
  containers:
  - name: sleepr
    image: busybox
    command:
    - /bin/sh
    - -c
    - sleep 24000
'''


#    fromYaml('''
#kind: Pod
#apiVersion: v1
#metadata:
#  name: dummy-pod
#  labels:
#    blow: job
#spec:
#  containers:
#  - name: sleepr
#    image: busybox
#    command:
#    - /bin/sh
#    - -c
#    - sleep 24000
#''',k8s_client)

    fromYaml('''
kind: NetworkChaos
apiVersion: chaos-mesh.org/v1alpha1
metadata:
  name: web-show-network-delay
spec:
  action: delay
  mode: one
  selector:
    namespaces:
      - default
    labelSelectors:
      "app": "web-show"
  delay:
    latency: "10ms"
  duration: "30s"
  scheduler:
    cron: "@every 60s"
''',k8s_client)




def fromYaml(rawData, client=None, **kwargs):
    ''' invoke the K8s API to create or replace an object given as YAML spec.
        @param rawData: either a string or an opened input stream with a
                        YAML formatted spec, as you'd use for `kubectl apply -f`
        @param client: (optional) preconfigured client environment to use for invocation
        @param kwargs: (optional) further arguments to pass to the create/replace call
        @return: response object from Kubernetes API call
    '''

    for obj in yaml.load_all(rawData):
        createOrUpdateOrReplace(obj, client, **kwargs)


def createOrUpdateOrReplace(obj, client=None, **kwargs):
    ''' invoke the K8s API to create or replace a kubernetes object.
        The first attempt is to create(insert) this object; when this is rejected because
        of an existing object with same name, we attempt to patch this existing object.
        As a last resort, if even the patch is rejected, we *delete* the existing object
        and recreate from scratch.
        @param obj: complete object specification, including API version and metadata.
        @param client: (optional) preconfigured client environment to use for invocation
        @param kwargs: (optional) further arguments to pass to the create/replace call
        @return: response object from Kubernetes API call
    '''
    k8sApi = findK8sApi(obj, client)
    try:
        res = invokeApi(k8sApi, 'create', obj, **kwargs)
        logging.debug('K8s: %s created -> uid=%s', describe(obj), res.metadata.uid)
    except kubernetes.client.rest.ApiException as apiEx:
        if apiEx.reason != 'Conflict': raise
        try:
            # asking for forgiveness...
            res = invokeApi(k8sApi, 'patch', obj, **kwargs)
            logging.debug('K8s: %s PATCHED -> uid=%s', describe(obj), res.metadata.uid)
        except kubernetes.client.rest.ApiException as apiEx:
            if apiEx.reason != 'Unprocessable Entity': raise
            try:
                # second attempt... delete the existing object and re-insert
                logging.debug('K8s: replacing %s FAILED. Attempting deletion and recreation...', describe(obj))
                res = invokeApi(k8sApi, 'delete', obj, **kwargs)
                logging.debug('K8s: %s DELETED...', describe(obj))
                res = invokeApi(k8sApi, 'create', obj, **kwargs)
                logging.debug('K8s: %s CREATED -> uid=%s', describe(obj), res.metadata.uid)
            except Exception as ex:
                message = 'K8s: FAILURE updating %s. Exception: %s' % (describe(obj), ex)
                logging.error(message)
                raise RuntimeError(message)
    return res


def patchObject(obj, client=None, **kwargs):
    k8sApi = findK8sApi(obj, client)
    try:
        res = invokeApi(k8sApi, 'patch', obj, **kwargs)
        logging.debug('K8s: %s PATCHED -> uid=%s', describe(obj), res.metadata.uid)
        return res
    except kubernetes.client.rest.ApiException as apiEx:
        if apiEx.reason == 'Unprocessable Entity':
            message = 'K8s: patch for %s rejected. Exception: %s' % (describe(obj), apiEx)
            logging.error(message)
            raise RuntimeError(message)
        else:
            raise


def deleteObject(obj, client=None, **kwargs):
    k8sApi = findK8sApi(obj, client)
    try:
        res = invokeApi(k8sApi, 'delete', obj, **kwargs)
        logging.debug('K8s: %s DELETED. uid was: %s', describe(obj), res.details and res.details.uid or '?')
        return True
    except kubernetes.client.rest.ApiException as apiEx:
        if apiEx.reason == 'Not Found':
            logging.warning('K8s: %s does not exist (anymore).', describe(obj))
            return False
        else:
            message = 'K8s: deleting %s FAILED. Exception: %s' % (describe(obj), apiEx)
            logging.error(message)
            raise RuntimeError(message)



def findK8sApi(obj, client=None):
    ''' Investigate the object spec and lookup the corresponding API object
        @param client: (optional) preconfigured client environment to use for invocation
        @return: a client instance wired to the apriopriate API
    '''
    grp, _, ver = obj['apiVersion'].partition('/')
    if ver == '':
        ver = grp
        grp = 'core'
    # Strip 'k8s.io', camel-case-join dot separated parts. rbac.authorization.k8s.io -> RbacAuthorzation
    grp = ''.join(part.capitalize() for part in grp.rsplit('.k8s.io', 1)[0].split('.'))
    ver = ver.capitalize()

    k8sApi = '%s%sApi' % (grp, ver)
    return getattr(kubernetes.client, k8sApi)(client)


def invokeApi(k8sApi, action, obj, **args):
    ''' find a suitalbe function and perform the actual API invocation.
        @param k8sApi: client object for the invocation, wired to correct API version
        @param action: either 'create' (to inject a new objet) or 'replace','patch','delete'
        @param obj: the full object spec to be passed into the API invocation
        @param args: (optional) extraneous arguments to pass
        @return: response object from Kubernetes API call
    '''
    # transform ActionType from Yaml into action_type for swagger API
    kind = camel2snake(obj['kind'])
    # determine namespace to place the object in, supply default

    try: namespace = obj['metadata']['namespace']

    except: namespace = 'default'

    functionName = '%s_%s' %(action,kind)
    if hasattr(k8sApi, functionName):
        # namespace agnostic API
        function = getattr(k8sApi, functionName)
    else:
        functionName = '%s_namespaced_%s' %(action,kind)
        function = getattr(k8sApi, functionName)
        args['namespace'] = namespace
    if not 'create' in functionName:
        args['name'] = obj['metadata']['name']
    if 'delete' in functionName:
        from kubernetes.client.models.v1_delete_options import V1DeleteOptions
        obj = V1DeleteOptions()

    return function(body=obj, **args)


def describe(obj):
    return "%s '%s'" % (obj['kind'], obj['metadata']['name'])


def camel2snake(string):
    string = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', string)
    string = re.sub('([a-z0-9])([A-Z])', r'\1_\2', string).lower()
    return string


if __name__=='__main__':
    print("yey")
    runUsageExample()


# Chaos Mesh Go Library

Chaos Mesh can be triggered from a go library - investigate this as a last resort.